# Определение пола человека по его имени
Работу выполнили студентки 2 курса Высшей школы ИТИС Грибанова Анастасия и Шафеева Дина, группа 11-803.

**Цель проекта:** научиться по имени, фамилии и отчеству человека определять его пол. 

Для достижения этой цели берутся данные 62609 человек и на их основе строятся различные модели.
Для работы берутся данные русскоязычного населения, из-за наличия у тех отчества, которое будет являться дополнительным параметром для анализа.

**Поля:** полями данных являются полные имена, а именно: фамилия, имя, отчество человека. Последнее поле в таблице - пол.

**Источник:** https://github.com/Rai220/MlSexDetector/blob/master/dataMini.csv

In [1]:
import chardet
import pandas as pd
from statistics import mode

with open('C:/Users/Nastya/dataMini.csv', 'rb') as f:
    result = chardet.detect(f.readline()) 


data = pd.read_csv('C:/Users/Nastya/dataMini.csv', encoding=result['encoding'])
data.head()

,ФАМИЛИЯ,ИМЯ,ОТЧЕСТВО,ПОЛ
0,АБАБКОВ,СЕРГЕЙ,ИВАНОВИЧ,М
1,АБАЕВ,АНДРЕЙ,АЛЕКСАНДРОВИЧ,М
2,АБАЗОВА,ЕЛЕНА,НИКОЛАЕВНА,Ж
3,АБАКУМОВ,СЕРГЕЙ,ВАСИЛЬЕВИЧ,М
4,АБАКУМОВ,СЕРГЕЙ,ВАСИЛЬЕВИЧ,М


In [2]:
data.shape

(62609, 4)

Узнаем, есть ли в данных пропущенные значения, с помощью функции isnull()

In [3]:
data.isnull().any().any()

True

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62609 entries, 0 to 62608
Data columns (total 4 columns):
ФАМИЛИЯ     62609 non-null object
ИМЯ         62609 non-null object
ОТЧЕСТВО    62579 non-null object
ПОЛ         62609 non-null object
dtypes: object(4)
memory usage: 1.9+ MB


Воспользовавшись функцией info(), выяснили, что пропуски встречаются только в столбце «Отчество», и их не так много, всего 30, так что можем заменить их на самое часто встречающееся значение в этом столбце - моду. Найдем это значение.

In [5]:
print(data.mode()['ОТЧЕСТВО'])

0    АЛЕКСАНДРОВНА
Name: ОТЧЕСТВО, dtype: object


Подставляем эту моду ('Александровна') в пропущенные ячейки.

In [6]:
data = data.fillna('АЛЕКСАНДРОВНА')
print(data)

         ФАМИЛИЯ     ИМЯ       ОТЧЕСТВО ПОЛ
0        АБАБКОВ  СЕРГЕЙ       ИВАНОВИЧ   М
1          АБАЕВ  АНДРЕЙ  АЛЕКСАНДРОВИЧ   М
2        АБАЗОВА   ЕЛЕНА     НИКОЛАЕВНА   Ж
3       АБАКУМОВ  СЕРГЕЙ     ВАСИЛЬЕВИЧ   М
4       АБАКУМОВ  СЕРГЕЙ     ВАСИЛЬЕВИЧ   М
...          ...     ...            ...  ..
62604     ЖАРОВА  МАРИНА    АНАТОЛЬЕВНА   Ж
62605  СИРОТКИНА  ГАЛИНА     НИКОЛАЕВНА   Ж
62606    ЧУВИЛИН  СЕРГЕЙ     ВИКТОРОВИЧ   М
62607    БЕКАСОВ  МИХАИЛ     НИКОЛАЕВИЧ   М
62608     ЛЕЗОВА    ИННА     НИКОЛАЕВНА   Ж

[62609 rows x 4 columns]


Еще раз проверим наши данные на наличие пропущенных значений с помощью той же функции isnull().

In [7]:
data.isnull().any().any()

False

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62609 entries, 0 to 62608
Data columns (total 4 columns):
ФАМИЛИЯ     62609 non-null object
ИМЯ         62609 non-null object
ОТЧЕСТВО    62609 non-null object
ПОЛ         62609 non-null object
dtypes: object(4)
memory usage: 1.9+ MB


Пропущенные данные успешно заполнились модой